<a href="https://colab.research.google.com/github/PritamGoyal/shoplifting/blob/main/shoplifting_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:


source_dir = "/content/drive/MyDrive/Shoplifting"

destination_dir = "/content/drive/MyDrive/All_Video_Clips"

if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            shutil.copy(file_path, destination_dir)
            print(f"Copied {file} to {destination_dir}")

print("All video clips copied successfully.")


Copied Shoplifting054_x264_16.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_0.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_19.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_24.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_15.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_13.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_21.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_10.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_23.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_12.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_11.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_18.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplifting054_x264_1.mp4 to /content/drive/MyDrive/All_Video_Clips
Copied Shoplif

In [2]:
csv_file_path = "/content/Shoplifting.csv"

custom_headers = ["video", "type", "label"]

df = pd.read_csv(csv_file_path, header=None, names=custom_headers)

print(df.head())

                   video         type  label
0  Shoplifting001_x264_0  Shoplifting      0
1  Shoplifting001_x264_1  Shoplifting      0
2  Shoplifting001_x264_2  Shoplifting      0
3  Shoplifting001_x264_3  Shoplifting      0
4  Shoplifting001_x264_4  Shoplifting      0


In [3]:
df['type'].value_counts()

type
Shoplifting    896
Name: count, dtype: int64

In [4]:
df.drop(columns=['type'], inplace=True)

In [5]:
df.head()

,video,label
0,Shoplifting001_x264_0,0
1,Shoplifting001_x264_1,0
2,Shoplifting001_x264_2,0
3,Shoplifting001_x264_3,0
4,Shoplifting001_x264_4,0


In [6]:
df

,video,label
0,Shoplifting001_x264_0,0
1,Shoplifting001_x264_1,0
2,Shoplifting001_x264_2,0
3,Shoplifting001_x264_3,0
4,Shoplifting001_x264_4,0
...,...,...
891,Shoplifting054_x264_27,0
892,Shoplifting054_x264_28,0
893,Shoplifting054_x264_29,0
894,Shoplifting054_x264_30,0


In [7]:
import cv2
import numpy as np
from tqdm import tqdm

# Define function to preprocess videos and extract features
def preprocess_videos(df):
    features = []
    labels = []

    # Loop through each row in the DataFrame
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Videos"):
        video_path = "/content/drive/MyDrive/All_Video_Clips/" + row['video'] + ".mp4"
        label = row['label']

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Initialize variables to store features for each video
        video_features = []

        # Process each frame in the video
        while True:
            ret, frame = cap.read()

            # If there are no more frames, break the loop
            if not ret:
                break

            # Resize frame to a suitable size for feature extraction (e.g., 224x224)
            resized_frame = cv2.resize(frame, (224, 224))

            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

            # Extract features from the frame (e.g., using Histogram of Oriented Gradients)
            # Replace this with your preferred feature extraction method
            # Here's a placeholder feature extraction method:
            hog = cv2.HOGDescriptor()
            features_vector = hog.compute(gray_frame)
            video_features.append(features_vector)

        # Close the video file
        cap.release()

        # Aggregate features for the entire video (e.g., by taking the mean)
        aggregated_features = np.mean(video_features, axis=0)

        # Append aggregated features and label to the lists
        features.append(aggregated_features)
        labels.append(label)

    return np.array(features), np.array(labels)

# Call the preprocess_videos function with your DataFrame
features, labels = preprocess_videos(df)


Processing Videos: 100%|██████████| 896/896 [11:31<00:00,  1.30it/s]


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Initialize and train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = svm_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.9333333333333333
Precision: 0.9285714285714286
Recall: 0.7222222222222222
F1 Score: 0.8125000000000001
